In [3]:
# 用窗口取数, 
import os
import pandas as pd
import numpy as np

# 参数设置
window_size = 28
step_size = 14

# 读取数据
input_file = '../DATA/3_labeled_tactile_dataset_clean.csv'
output_file = '../DATA/4_labeled_window_dataset.csv'
output_file_normalization = '../DATA/4_labeled_window_dataset_Normalization.csv'

df = pd.read_csv(input_file)

# 初始化存储窗口信息的列表
windowed_data = []

# 初始化 window_id
window_id = 0

normalization = True


# 循环以 step_size 步长遍历数据
for start in range(0, len(df) - window_size + 1, step_size):
    end = start + window_size

    window = df.iloc[start:end].copy()

    if normalization:
        cols_to_normalize = [
            'e0', 'e1', 'e2', 'e3', 'e4', 'e5', 'e6', 
            'de0', 'de1', 'de2', 'de3', 'de4', 'de5', 'de6', 
            'tau_J0', 'tau_J1', 'tau_J2', 'tau_J3', 'tau_J4', 'tau_J5', 'tau_J6', 
            'tau_ext0', 'tau_ext1', 'tau_ext2', 'tau_ext3', 'tau_ext4', 'tau_ext5', 'tau_ext6'
        ]
        
        # 对每一列进行归一化
        for col in cols_to_normalize:
            mean_col = np.mean(window[col])
            std_col = np.std(window[col])
            window[col] = (window[col] - mean_col) / (std_col + 1e-5)

    # 生成 window_id
    window['window_id'] = window_id
    window_id += 1  # 每次迭代增加 window_id
    
    # 判断窗口内 touch_type 是否全为 'NC'
    unique_touch_types = window['touch_type'].unique()
    if len(unique_touch_types) == 1 and unique_touch_types[0] == 'NC':
        window_touch_type = 'NC'
    else:
        # 如果不是全为 'NC'，取窗口内唯一的非 'NC' 的 touch_type
        non_nc_touch_types = [t for t in unique_touch_types if t != 'NC']
        window_touch_type = non_nc_touch_types[0] if non_nc_touch_types else 'NC'
    
    # 给窗口内所有行赋值 window_touch_type
    window['window_touch_type'] = window_touch_type
    
    # 添加窗口数据到列表
    windowed_data.append(window)

# 合并所有窗口数据
windowed_df = pd.concat(windowed_data, ignore_index=True)

if normalization:
    # 删除已存在的文件
    if os.path.exists(output_file_normalization):
        os.remove(output_file_normalization)

    # 保存到新的CSV文件
    windowed_df.to_csv(output_file_normalization, index=False)

else:

    if os.path.exists(output_file):
        os.remove(output_file)

    # 保存到新的CSV文件
    windowed_df.to_csv(output_file, index=False)
    

print(f"{output_file if not normalization else output_file_normalization} 文件已生成，总行数 = {len(windowed_df)}")


../DATA/4_labeled_window_dataset_Normalization.csv 文件已生成，总行数 = 244692


In [5]:
dt = pd.read_csv('../DATA/4_labeled_window_dataset.csv')
dt.head()
# # dt[dt['window_touch_type']=='ST']['window_id'].nunique()
# grouped = dt.groupby(['window_id', 'window_touch_type', 'touch_type']).size().reset_index(name='count')
# # # 展示结果
# print(grouped)
# # dt[dt['window_id']==0]
# # print(len(dt)/28)

,index,time,tau_J0,tau_J1,tau_J2,tau_J3,tau_J4,tau_J5,tau_J6,tau_J_d0,...,etau_J2,etau_J3,etau_J4,etau_J5,etau_J6,label,block_id,touch_type,window_id,window_touch_type
0,755,3.770009,-0.264001,-33.450733,-1.669200,19.267954,0.492881,2.457165,-0.018444,0.0,...,1.669200,-19.267954,-0.492881,-2.457165,0.018444,1,0.0,DT,0,DT
1,756,3.775012,-0.264001,-33.415028,-1.704905,19.267954,0.483726,2.468151,-0.069713,0.0,...,1.704905,-19.267954,-0.483726,-2.468151,0.069713,1,0.0,DT,0,DT
2,757,3.780020,-0.264001,-33.415028,-1.704905,19.267954,0.483726,2.468151,-0.069713,0.0,...,1.704905,-19.267954,-0.483726,-2.468151,0.069713,1,0.0,DT,0,DT
3,758,3.785017,-0.410791,-34.216419,-1.815989,19.867014,1.125511,4.366039,-0.141125,0.0,...,1.815989,-19.867014,-1.125511,-4.366039,0.141125,1,0.0,DT,0,DT
4,759,3.790012,-0.410791,-34.216419,-1.815989,19.867014,1.125511,4.366039,-0.141125,0.0,...,1.815989,-19.867014,-1.125511,-4.366039,0.141125,1,0.0,DT,0,DT


In [6]:
dt = pd.read_csv('../DATA/4_labeled_window_dataset_Normalization.csv')
dt.head()

,index,time,tau_J0,tau_J1,tau_J2,tau_J3,tau_J4,tau_J5,tau_J6,tau_J_d0,...,etau_J2,etau_J3,etau_J4,etau_J5,etau_J6,label,block_id,touch_type,window_id,window_touch_type
0,755,3.770009,0.850883,0.928364,0.851350,-0.984347,-1.130154,-1.190184,1.294259,0.0,...,1.669200,-19.267954,-0.492881,-2.457165,0.018444,1,0.0,DT,0,DT
1,756,3.775012,0.850883,0.934080,0.831814,-0.984347,-1.175528,-1.174969,-0.023532,0.0,...,1.704905,-19.267954,-0.483726,-2.468151,0.069713,1,0.0,DT,0,DT
2,757,3.780020,0.850883,0.934080,0.831814,-0.984347,-1.175528,-1.174969,-0.023532,0.0,...,1.704905,-19.267954,-0.483726,-2.468151,0.069713,1,0.0,DT,0,DT
3,758,3.785017,0.759591,0.805776,0.771036,-0.847992,2.005237,1.453563,-1.859026,0.0,...,1.815989,-19.867014,-1.125511,-4.366039,0.141125,1,0.0,DT,0,DT
4,759,3.790012,0.759591,0.805776,0.771036,-0.847992,2.005237,1.453563,-1.859026,0.0,...,1.815989,-19.867014,-1.125511,-4.366039,0.141125,1,0.0,DT,0,DT


In [9]:
print(dt.groupby('window_touch_type')['window_id'].nunique())

window_touch_type
DT     839
G     1633
NC    4233
P     1571
ST     463
Name: window_id, dtype: int64
